## Train Named Entity Recognition model with SpaCy
This project shows how to extract information from text documents using transfer learning with pretrained model from SpaCy library.


In [1]:
# import ibraries
from spacy.util import filter_spans
import json
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [2]:
with open('samples.json', 'r') as f:
    data = json.load(f)
    
print(data[0])

{'id': 6302, 'text': 'Yes, it is a product', 'label': [[13, 20, 'PRODUCT']], 'Comments': []}


### Prepare training data

In [3]:
training_data = {
    'classes' : ['PRODUCT', "SERVICE", "PROCESS"],
    'annotations' : []
}

for example in data:
  data_row = {}
  data_row['text'] = example['text']
  data_row['entities'] = []

  for annotation in example['label']:
    start = annotation[0]
    end = annotation[1]
    label = annotation[2]
    data_row['entities'].append((start, end, label))
  training_data['annotations'].append(data_row)
  
print(training_data['annotations'][1])

{'text': 'Yes, it is a service', 'entities': [(13, 20, 'SERVICE')]}


In [4]:
nlp = spacy.blank("en") 

doc_bin = DocBin()

In [5]:
for training_row  in tqdm(training_data['annotations']): 
    text = training_row['text']
    labels = training_row['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

    
doc_bin.to_disk("training.spacy")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 5159.05it/s]


### Run commands for training model

In [6]:
!python -m spacy init fill-config base_config.cfg config.cfg

!python -m spacy train config.cfg --output ./ --paths.train ./training.spacy --paths.dev ./training.spacy

/home/michal/anaconda3/envs/ner_model_training_spacy/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/michal/anaconda3/envs/ner_model_training_spacy/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
/home/michal/anaconda3/envs/ner_model_training_spacy/lib/python3.10/site-packages/transforme

### Test model

In [7]:
nlp_ner = spacy.load("model-best")

doc = nlp_ner("It’s a complicated question, but let’s start with a product. It’s a complicated question, but let’s start with a service. It’s a complicated question, but let’s start with a process")

colors = {"PRODUCT": "#F67DE3", "MSERVICE": "#7DF6D9", "PROCESS":"#FFFFFF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)